#Ingest data from landing to bronze using Autoloader

We access to the storage account and set the paths to the files

In [0]:
###Establish account key G2
spark.conf.set("fs.azure.account.key.tfmgendos.dfs.core.windows.net","QT/ULR9KopMmUZmz55avGjsJf/cuoAdFd/HUT3qDxs5pLllRhvXwyQEsqn8b1RKn91hv7lTuhb9K+AStoUFp+A==")

basepath="abfss://tfm@tfmgendos.dfs.core.windows.net/"

dbutils.fs.ls(basepath)


Out[1]: [FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/bronze/', name='bronze/', size=0, modificationTime=1662129140000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1662129150000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/', name='landing/', size=0, modificationTime=1662129135000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1662129145000)]

In [0]:

# autoloader table and checkpoint paths
WEOByCountryGroup_Bronce = basepath + "bronze/fmi/WEOByCountryGroup/"
bronzeCheckpoint = basepath + "bronze/fmi/WEOByCountryGroup/checkpoint/"
bronzeSchema = basepath + "bronze/fmi/WEOByCountryGroup/schema/"

# Landing Zone
landingZoneLocation = basepath + "landing/fmi/weobycountrygroup/"





In [0]:
dbutils.fs.ls(landingZoneLocation)

Out[3]: [FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/fmi/weobycountrygroup/31be0d2d-91ce-4457-8cd1-86c90c519c97.csv', name='31be0d2d-91ce-4457-8cd1-86c90c519c97.csv', size=595535, modificationTime=1662576542000)]

In [0]:
# "cloudFiles" indicates the use of Auto Loader
# We are using csv and give a schema location so that the current schema can be saved and infered easily
dfBronze = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("cloudFiles.schemaLocation", bronzeSchema) \
  .option("header", "true") \
  .option("delimiter", "\t") \
  .load(landingZoneLocation)




In [0]:
from pyspark.sql import functions as F

#Replace the space character in column names to "_"
dfBronze = dfBronze.select([F.col(col).alias(col.replace(' ', '_')) for col in dfBronze.columns])


In [0]:
#
dfBronze.columns


Out[6]: ['WEO_Country_Group_Code',
 'WEO_Subject_Code',
 'Country_Group_Name',
 'Subject_Descriptor',
 'Subject_Notes',
 'Units',
 'Scale',
 'Country/Series-specific_Notes',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024',
 '2025',
 '2026',
 '2027',
 'Estimates_Start_After',
 '_rescued_data']

In [0]:
# The stream will shut itself off when it is finished based on the trigger once feature
# The checkpoint location saves the state of the ingest when it is shut off so we know where to pick up next time
dfBronze.writeStream \
  .format("delta") \
  .trigger(once=True) \
  .option("checkpointLocation", bronzeCheckpoint) \
  .option("header", "true")\
  .start(WEOByCountryGroup_Bronce)

Out[7]: <pyspark.sql.streaming.StreamingQuery at 0x7fb1600ea280>

In [0]:
#Show data
#Load and verify that it's been saved properly to bronze
dfBronze = spark.read.format("delta").load(WEOByCountryGroup_Bronce)

dfBronze.show()

+----------------------+----------------+------------------+--------------------+--------------------+--------------------+--------+-----------------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------------------+--------------------+
WEO_Country_Group_Code|WEO_Subject_Code|Country_Group_Name| Subject_Descriptor| Subject_Notes| Units| Scale|Country/Series-specific_Notes| 1980| 1981| 1982| 1983| 1984| 1985| 1986| 1987| 1988| 1989| 1990| 1991| 1992| 1993| 1994| 1995| 1996| 1997| 1998| 1999| 2000| 2001| 2002| 2003| 2004| 2005| 2006| 2007| 2008| 2009| 2010| 2011| 2012| 2013| 2014| 2015| 2016| 2017| 2018| 2019| 2020| 2021| 2022| 2023| 2024| 2025| 2026| 2027|Estimates_Start_After| _rescued_data|
+----------------------+----------------+------------------+--------------------+--------------------+--------------------+--------+-----------------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------------------+--------------------+
 001| NGDP_RPCH| World|Gross domestic pr...| null| Percent change| null| null| 2.072| 1.944| 0.683| 2.654| 4.579| 3.650| 3.628| 3.871| 4.676| 3.805| 3.426| 2.685| 2.331| 2.041| 3.230| 3.359| 3.926| 4.013| 2.577| 3.547| 4.799| 2.446| 2.902| 4.256| 5.422| 4.857| 5.421| 5.559| 3.051| -0.086| 5.432| 4.285| 3.572| 3.415| 3.520| 3.424| 3.251| 3.747| 3.612| 2.865| -3.063| 6.108| 3.585| 3.551| 3.404| 3.378| 3.315| 3.263| null|{"_c57":"","_file...|
 001| NGDP_RPCHMK| World|Gross domestic pr...| null|Percent change (m...| null| null| 1.648| 2.010| 0.456| 2.801| 4.833| 3.681| 3.383| 3.762| 4.577| 3.805| 3.150| 1.965| 2.535| 1.422| 3.157| 2.925| 3.343| 3.502| 2.341| 3.162| 4.218| 1.672| 1.897| 2.857| 4.035| 3.559| 3.948| 3.837| 1.576| -2.009| 4.177| 3.074| 2.507| 2.592| 2.811| 2.832| 2.602| 3.247| 3.151| 2.487| -3.456| 5.805| 3.489| 3.115| 2.819| 2.827| 2.785| 2.736| null|{"_c57":"","_file...|
 001| NGDPD| World|Gross domestic pr...| null| U.S. dollars|Billions| null|11,237.562|11,493.502|11,281.394|11,583.895|11,997.410|12,523.328|14,771.979|17,008.945|19,125.335|20,122.142|23,654.018|24,492.481|25,328.261|26,039.542|27,985.290|31,199.710|32,068.872|31,979.647|31,824.477|32,949.687|34,039.874|33,788.307|34,900.086|39,193.343|44,116.922|47,792.249|51,762.962|58,443.072|64,108.440|60,731.151|66,435.818|73,747.749| 75,116.959| 77,290.820| 79,321.720| 75,042.630| 76,267.375| 80,945.405| 86,084.955| 87,536.338| 85,238.623| 96,292.574|103,867.109|110,751.392|117,007.745|123,297.350|129,728.878|136,383.604| null|{"_c57":"","_file...|
 001| PPPGDP| World|Gross domestic pr...| null|Purchasing power ...|Billions| null|13,393.156|14,919.801|15,896.804|16,920.524|18,305.638|19,562.438|20,648.951|21,959.554|23,764.986|25,605.078|27,632.847|29,269.241|33,243.349|34,693.190|36,478.405|38,608.597|40,826.484|43,211.132|44,893.369|47,091.180|50,461.918|52,835.180|55,175.355|58,541.865|63,283.578|68,372.360|74,217.901|80,339.738|84,267.283|84,476.604|90,032.782|95,542.742|100,530.290|105,453.395|109,349.957|111,568.483|115,863.055

In [0]:
# check rows number
dfBronze.count()


Out[9]: 1665